In [1]:
import numpy as np
import math
import copy

In [2]:
from sklearn import model_selection, datasets, svm
from sklearn.preprocessing import normalize

In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

In [4]:
iris = datasets.load_iris()
X = iris.data
# X = normalize(iris.data)
Y = iris.target
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.33, random_state=42)

In [5]:
N = 4

In [6]:
def feature_map(X):
    q = QuantumRegister(N, name='q')
    c = ClassicalRegister(N, name='c1')
    qc = QuantumCircuit(q, c)
    
    for i, x in enumerate(X):
        qc.rx(x, i) 
        
    return qc, c

In [7]:
def variational_circuit(qc, theta):
    for i in range(N-1):
        qc.cx(i, i+1) # cnot gate

    qc.cx(N-1, 0)
    qc.barrier()
    for i in range(N):
        qc.ry(theta[i], i)

    qc.barrier()
    
    return qc

In [8]:
def binary2int(bstring: str):
    if not isinstance(bstring, str):
        bstring = str(bstring)
    return int(bstring, 2)

# print(binary2int("101101"))

In [9]:
def get_max(dict_to_verify: dict):
    return max(dict_to_verify, key=dict_to_verify.get)

In [10]:
def quantum_nn(X, theta, simulator=True):
    qc, c = feature_map(X)
    
    qc.barrier()
    qc = variational_circuit(qc, theta)
    
    for i in range(N):
        qc.measure(i, c[i])
    
    
    shots = 1E4
    results = AerSimulator().run(qc, shots=1E4, memory=True).result()
    counts = results.get_counts(qc)

    stats = {}
    for key, value in counts.items():
        stats[binary2int(key)] = value / shots

    result = get_max(stats)
    
    return result, stats[result]

quantum_nn(X[0], np.random.rand(N))

(15, 0.2512)

In [11]:
def loss(prediction, target):
    # print(f"loss: {math.pow(target-prediction, 2)}")
    return math.pow(target-prediction, 2)

In [12]:
def gradient(X, Y, theta):
    delta = 0.01
    grad = []
    for i in range(len(theta)):
        dtheta = copy.copy(theta)
        dtheta[i] += delta

        int_pred1, value_pred1 = quantum_nn(X, dtheta)
        int_pred2, value_pred2 = quantum_nn(X, theta)

        # print(f"{i}\tpred1: {int_pred1}, {value_pred1}\n \tpred2: {int_pred2}, {value_pred2}")
        grad.append((loss(value_pred1, Y) - loss(value_pred2, Y))/delta)

    return np.array(grad) 

gradient(X_train[0], Y_train[0], np.random.rand(N))

array([-0.380856,  0.380072, -0.471905,  0.163176])

In [13]:
def accuracy(X, Y, theta):
    counter = 0 
    for X_i, Y_i in zip(X, Y):
        pred_int, pred_value = quantum_nn(X_i, theta)

        if pred_int == Y_i:
            counter += 1

    return counter / len(Y)

accuracy(X_test, Y_test, np.random.rand(N))

0.0

In [15]:
pred_int, pred_value = quantum_nn(X_train[5], np.random.rand(N))
target = Y_train[5]
print(pred_int, target)

2 1


In [16]:
loss(pred_int, target)

1.0

In [25]:
eta = 0.05
loss_list = []
theta = np.random.rand(N)
print(theta)

[0.42981369 0.97045091 0.13767134 0.24298117]


In [26]:
for i in range(100):
    
    loss_tmp = []
    for X_i, Y_i in zip(X_train, Y_train):
        pred_int, pred_value = quantum_nn(X_i, theta)
        loss_tmp.append(loss(pred_int, Y_i))
        theta = theta - eta * gradient(X_i, Y_i, theta) # gradiente descendent

    
    loss_list.append(np.mean(loss_tmp))
    acc = accuracy(X_train, Y_train, theta)

    print(f"i={i+1:2}\t loss={loss_list[-1]:.3f}\t acc={acc}\n \ttheta={theta}")

i= 1	 loss=47.160	 acc=0.0
 	theta=[-0.06811056  0.15388686 -0.24304901 -0.42497268]
i= 2	 loss=65.040	 acc=0.0
 	theta=[-0.08740776 -0.54508494  0.32670964  0.06827537]
i= 3	 loss=59.400	 acc=0.0
 	theta=[-0.46101211 -0.47405664 -0.48679866 -0.61374138]
i= 4	 loss=64.510	 acc=0.0
 	theta=[-0.91411546  0.02469571 -0.20614116 -0.32726108]
i= 5	 loss=65.790	 acc=0.0
 	theta=[ 0.27615829 -0.28004074 -0.89431056 -0.39111593]
i= 6	 loss=73.950	 acc=0.0
 	theta=[ 0.46877824 -0.60971654 -0.91573921  0.29416027]
i= 7	 loss=77.920	 acc=0.0
 	theta=[ 0.18031514 -0.06435309 -1.22217056 -0.35588818]
i= 8	 loss=58.800	 acc=0.06
 	theta=[-0.60501421  0.44183871 -1.65980506  0.03368307]
i= 9	 loss=64.340	 acc=0.0
 	theta=[-0.04467936  0.16235756 -1.10925591  0.23513572]
i=10	 loss=69.810	 acc=0.0
 	theta=[-0.16807501  0.12058001 -0.75146476  0.21645657]
i=11	 loss=67.350	 acc=0.0
 	theta=[ 0.48231589 -0.37543329 -1.29934206 -0.10328758]
i=12	 loss=74.930	 acc=0.0
 	theta=[ 0.17472629 -0.61558289 -0.7